In [3]:
import pandas as pd 
import numpy as np 
import tweepy


In [16]:
# importing the data -- it should be stored in a dirrectory called data
politifact_fake = pd.read_csv('data/politifact_fake.csv')
politifact_real = pd.read_csv('data/politifact_real.csv')

gossipcop_fake = pd.read_csv('data/gossipcop_fake.csv')
gossipcop_real = pd.read_csv('data/gossipcop_real.csv')

## looking at shape and columns 

In [17]:
# looking at shape and columns 
print("politifact fake:" + str(politifact_fake.shape)) # 432 pieces of news  (fake)
print("politifact real:" + str(politifact_real.shape)) # 624 pieces of news (real)

print("gossip cop fake" + str(gossip_cop_fake.shape)) # 5323 pieces of news (fake)
print("gossip cop real" + str(gossip_cop_real.shape)) # 16817 peices of news (real)

politifact fake:(432, 4)
politifact real:(624, 4)
gossip cop fake(4898, 4)
gossip cop real(15747, 4)


## drop na values 


In [18]:
# drop na values 
politifact_fake.dropna(inplace=True)
politifact_real.dropna(inplace=True)

gossip_cop_fake.dropna(inplace=True) 
gossip_cop_real.dropna(inplace=True)

## looking at shape and columns after dropping NA values 


In [19]:
# looking at shape and columns after dropping NA values 
print("politifact fake:" + str(politifact_fake.shape)) # 389 pieces of news  (fake)
print("politifact real:" + str(politifact_real.shape)) # 373 pieces of news (real)

print("gossip cop fake:" + str(gossip_cop_fake.shape)) # 4898 pieces of news (fake)
print("gossip cop real:" + str(gossip_cop_real.shape)) # 15747 pieces of news (real)

politifact fake:(389, 4)
politifact real:(373, 4)
gossip cop fake:(4898, 4)
gossip cop real:(15747, 4)


In [20]:
# column names
politifact_fake.columns

Index(['id', 'news_url', 'title', 'tweet_ids'], dtype='object')

## columns
- id is the id associated with a peice of news
- news_url is a link to the original news 
- title is the title associated with the news 
- tweet_ids is a tab seperated string of all the tweet ids associated with that piece of news

## converting the string of tweet ids to a list

In [21]:
politifact_fake['tweet_ids'] = politifact_fake.tweet_ids.str.split('\t')
politifact_real['tweet_ids'] = politifact_real.tweet_ids.str.split('\t')
gossipcop_fake['tweet_ids'] = gossipcop_fake.tweet_ids.str.split('\t')
gossipcop_real['tweet_ids'] = gossipcop_real.tweet_ids.str.split('\t')


## twitter API stuff

In [22]:
consumer_key = "SfH6LhmYyWltgdlTFuEm3ODvr"
consumer_secret = "WfzZyfovKaKuhjPV1EGGlTF9ha5zxuTGjIQIgdQ96GicdLyfxY"
access_token = "1311486910322286592-yXQQ8l6IPmexbdK5gBgGGyo3QL7ZVR"
access_token_secret = "U84aUFUgge2MLonznCKd3bodRedwT8bBi4NO6fAe2sHhP"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

## function for getting less than 100 tweets (API limit is 100 tweets at a time)

In [28]:
def get_tweets_small(row_data, number_of_tweets = False):
    # attributes destruuctured 
    news_id = row_data.id 
    news_url = row_data.news_url
    title = row_data.title
    id_list = row_data.tweet_ids
    # lists to store values in 
    tweet_ids_list = []
    username_list = []
    text_list = []
    news_id_list = []
    news_url_list = []
    title_list = []
    reply_list = []  
    if number_of_tweets:
        if (len(row_data.tweet_ids) > number_of_tweets + 1):
            id_list = row_data.tweet_ids[:number_of_tweets + 1]
    result = api.statuses_lookup(id_ = id_list)
    for status in result:
        # this is the ID for each individual tweet
        tweet_id = status.id
        tweet_ids_list.append(tweet_id)
        # username for each individual tweet
        username = status.user.screen_name 
        username_list.append(username)
        # text value for each individual tweet
        text = status.text
        text_list.append(text)
        # 
        reply = status.in_reply_to_status_id_str
        if reply is None:
            reply_list.append(False)
        else:
            reply_list.append(True)
        # these are repeated values: news_id, news_url and title will be the same for all tweets associated with a particular news story (row) 
        news_id_list.append(news_id)
        news_url_list.append(news_url)
        title_list.append(title)
    data = {"news_id": news_id_list, "news_url": news_url_list, "title": title_list, "tweet_id": tweet_ids_list, "username": username_list, "text": text_list, "reply": reply_list}
    return pd.DataFrame(data)





## function for getting more than 100 tweets

In [61]:
def get_tweets_large(row_data, number_of_tweets = False):

    # attributes destruuctured 
    news_id = row_data.id 
    news_url = row_data.news_url
    title = row_data.title
    id_list = row_data.tweet_ids
    # lists to store values in 
    tweet_ids_list = []
    username_list = []
    text_list = []
    news_id_list = []
    news_url_list = []
    title_list = []
    reply_list = []

    if number_of_tweets:
        if number_of_tweets < 100: # (len(row_data.tweet_ids) > number_of_tweets + 1) and
            i = 1
            tweet_subset = row_data.tweet_ids[:number_of_tweets + 1]
    else: 
        i = len(row_data.tweet_ids) // 100
        tweet_subset = row_data.tweet_ids[:99]

    for j in range(i):
        if number_of_tweets:
            tweet_subset = row_data.tweet_ids[:number_of_tweets + 1]
        else:
            tweet_subset = row_data.tweet_ids[j*100:j*100+99]
        result = api.statuses_lookup(tweet_subset)

        for status in result:
            # this is the ID for each individual tweet
            tweet_id = status.id
            tweet_ids_list.append(tweet_id)
            # username for each individual tweet
            username = status.user.screen_name 
            username_list.append(username)
            # text value for each individual tweet
            text = status.text
            text_list.append(text)
            # 
            reply = status.in_reply_to_status_id_str
            if reply is None:
                reply_list.append(False)
            else:
                reply_list.append(True)
            # these are repeated values: news_id, news_url and title will be the same for all tweets associated with a particular news story (row) 
            news_id_list.append(news_id)
            news_url_list.append(news_url)
            title_list.append(title)
    data = {"news_id": news_id_list, "news_url": news_url_list, "title": title_list, "tweet_id": tweet_ids_list, "username": username_list, "text": text_list, "reply": reply_list}
    return pd.DataFrame(data)

## function to aggregate all data in the data frame from 

In [62]:
def aggregate_data(df, name = "temp.csv", number_of_tweets = False):
    result = pd.DataFrame()
    for row in df.iterrows():
        row = row[1] # because iterrows() gives us a tuple (index, row) 
        if len(row.tweet_ids) < 100:
            df2 = get_tweets_small(row_data = row, number_of_tweets = number_of_tweets)
        else:
            df2 = get_tweets_large(row_data = row, number_of_tweets = number_of_tweets)
            
            
        # df2 = get_all_tweets_single_news_story(row_data = row, number_of_tweets = number_of_tweets)
        # print(row)
        result = result.append(df2, ignore_index= True)
    # result.to_csv(name)
    return result



In [66]:
# politifact_fake_test = politifact_fake.sample(3)

In [67]:
for row in politifact_fake_test.iterrows():
    print(len(row[1].tweet_ids))

31
61
163


In [68]:
aggregated_test_data = aggregate_data(politifact_fake_test)

In [70]:
aggregated_test_data.to_csv("test.csv")

In [71]:
aggregated_test_data.shape

(105, 7)

In [48]:
# test cases 
large_test = politifact_fake.iloc[2]
test_df_large_subset = get_tweets_large(large_test, number_of_tweets = 15)
test_df_large = get_tweets_large(large_test)

In [55]:
test_df_large.shape

(155, 7)

In [56]:
test_df_large_subset.shape

(12, 7)

In [49]:
test_df_large.shape

(155, 7)

In [58]:
small_test = politifact_fake.iloc[110]
test_df_small = get_tweets_small(small_test)
test_df_small_subset = get_tweets_small(small_test, number_of_tweets = 20)

In [59]:
test_df_small.shape

(48, 7)

In [60]:
test_df_small_subset.shape

(20, 7)

In [40]:
test_df.shape

(48, 7)